In [1]:
from guess_number import guess_number
from numpy import random, mean

def fix_len_num(num: int, length: int) -> str:
    num = str(num)
    return '0'*(length-len(num)) + num 

def average_int_result(func, repeat=10000, print_chart=False):
    """Decorator for 

    Args:
        func (_type_): _description_
        repeat (int, optional): _description_. Defaults to 10000.
        print_chart (bool, optional): _description_. Defaults to False.
    """
    def decorator(func):
        def dec_func(*args, **kwargs):
            results = [func(*args, **kwargs) for i in range(repeat)]
            avrg_res = round(mean(results))
            
            if print_chart:
                # Printing a simple chart in terminal
                min_res = min(results) # lower bound
                max_res = max(results) # upper bound
                results = {i: results.count(i)
                            for i in range(min_res, max_res+1)}
                for result in results.items():
                    max_val = max(results.values())
                    print(fix_len_num(result[0], 2), '-',
                            fix_len_num(result[1], 4), 
                            ('=' if result[0] == avrg_res else '-') 
                            * round(result[1]/max_val*30)) 
            return avrg_res
        return dec_func
    return decorator(func)

guess_number = average_int_result(guess_number, print_chart=True)
for hi in [10**i for i in range(2,6)]:
    print(f"\n1 <= number <= {hi}")
    print(f"Average number of tries is {guess_number(1, hi, bnr_srch=True)}")


1 <= number <= 100
01 - 0101 -
02 - 0215 --
03 - 0382 ---
04 - 0862 -------
05 - 1587 -------------
06 - 3173 ==========================
07 - 3680 ------------------------------
Average number of tries is 6

1 <= number <= 1000
01 - 0013 
02 - 0020 
03 - 0047 
04 - 0062 
05 - 0145 -
06 - 0317 --
07 - 0629 ----
08 - 1244 --------
09 - 2562 ===============
10 - 4961 ------------------------------
Average number of tries is 9

1 <= number <= 10000
01 - 0001 
02 - 0000 
03 - 0004 
04 - 0005 
05 - 0020 
06 - 0032 
07 - 0047 
08 - 0138 -
09 - 0265 --
10 - 0478 ---
11 - 1028 --------
12 - 2078 ===============
13 - 4099 ------------------------------
14 - 1805 -------------
Average number of tries is 12

1 <= number <= 100000
04 - 0002 
05 - 0001 
06 - 0005 
07 - 0006 
08 - 0008 
09 - 0021 
10 - 0038 
11 - 0105 -
12 - 0223 --
13 - 0450 ----
14 - 0801 -------
15 - 1662 ---------------
16 - 3299 =============================
17 - 3379 ------------------------------
Average number of tries is 16